In [15]:
from langchain_openai import ChatOpenAI

# Functions
def multiply(a:int, b:int):
    """Mutliply"""
    return a*b

def add(a:int, b:int):
    """Addition"""
    return a+b

tools = [multiply, add]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

In [20]:
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
        tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=MemorySaver())

In [25]:
from pprint import pprint
config = {"configurable":{"thread_id":"1"}}

input= {"messages": "Multiply 4*4"}

for event in graph.stream(input, config, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Multiply 4*4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_1TGNaNEqfzXdC4iM6AtXP5Ih)
 Call ID: call_1TGNaNEqfzXdC4iM6AtXP5Ih
  Args:
    a: 4
    b: 4
================================= Tool Message =================================
Name: multiply

16
================================== Ai Message ==================================

The result of multiplying 4 by 4 is 16.


In [ ]:
all_states = [s for s in graph.get_state_history(config)]
all_states

In [30]:
replay = all_states[-2]
replay.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efede6b-1b5c-6193-8000-6a21d85c71c5'}}

In [31]:
for event in graph.stream(None, replay.config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Multiply 4*4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_94SYZylb268nGqCDlO1LamTO)
 Call ID: call_94SYZylb268nGqCDlO1LamTO
  Args:
    a: 4
    b: 4
================================= Tool Message =================================
Name: multiply

16
================================== Ai Message ==================================

The result of multiplying 4 by 4 is 16.
